In [ ]:
import time, cv2, csv
import numpy as np
from PIL import Image
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from keras.models import load_model
from crawler import refine_coordinate
from crawler import crop_image
from preprocessBatch import preprocessing

full_image = "thsrc.jpg"
WIDTH = 140
HEIGHT = 48
FOLDER = "captcha/"
CSV_FILE = "auto-train.csv"

dic19 = {'2':0, '3':1, '4':2, '5':3, '7':4, '9':5, 'A':6, 'C':7, 'F':8, 'H':9, 'K':10, 'M':11, 'N':12, 'P':13, 'Q':14, 'R':15, 'T':16, 'Y':17, 'Z':18}
inv_dic19 = {v: k for k, v in dic19.items()}

print('model loading...')
model = load_model("thsrc_cnn_model.hdf5")

In [ ]:
i = 4000

while i < 5000:
    chrome_options = Options() # 啟動無頭模式
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    executable_path = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(executable_path=executable_path, options=chrome_options)
    driver.get('http://irs.thsrc.com.tw/IMINT/')

    driver.find_element_by_id('btn-confirm').click()
    time.sleep(1)

    driver.save_screenshot(full_image)

    element = driver.find_element_by_xpath('//*[@id="BookingS1Form_homeCaptcha_passCode"]')
    location = element.location
    size = element.size
    ratio = int(driver.execute_script('return window.devicePixelRatio'))

    coordinate = refine_coordinate(location, size, ratio)
    img = crop_image(coordinate)
    img = img.resize((WIDTH, HEIGHT), Image.ANTIALIAS)
    # convert rgba to rgb
    img = img.convert('RGB')
    img.save('captcha.jpg', "JPEG")

    preprocessing('captcha.jpg', 'preprocessing.jpg')

    train_data = np.stack([np.array(cv2.imread("preprocessing.jpg"))/255.0])
    prediction = model.predict(train_data)

    predict_captcha = ''
    for predict in prediction:
        value = np.argmax(predict[0])
        predict_captcha += inv_dic19[value]

    print(predict_captcha)

    # 填入表單
    startStation = Select(driver.find_element_by_name('selectStartStation'))
    startStation.select_by_value("1") # 台北

    selectDestinationStation = Select(driver.find_element_by_name('selectDestinationStation'))
    selectDestinationStation.select_by_value("2") # 台北

    toTimeTable = Select(driver.find_element_by_name('toTimeTable'))
    toTimeTable.select_by_index(8) # 09:00

    captchaInput = driver.find_element_by_name("homeCaptcha:securityCode")
    captchaInput.send_keys(predict_captcha) 

    driver.find_element_by_id('SubmitButton').click()
    time.sleep(1)
    if driver.title == "台灣高鐵網路訂位 > 選擇車次":
        # add to label
        i += 1
        print("success, add " + str(i))
        img.save(FOLDER + str(i) + ".jpg", "JPEG")
        with open(CSV_FILE,'a') as fd:
            fd.write(predict_captcha + "\n")

    driver.quit()

print("completed")